In [1]:
!del /Q utils\__pycache__

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
from utils.datasets import GhlKasperskyDataset, TepHarvardDataset, TepKasperskyDataset, SwatItrustDataset
from utils.metrics import time_span_metrics
from utils.watchmen import WatchSquad

# Preparing datasets

In [4]:
datasets = {
    0: GhlKasperskyDataset(),
    1: TepHarvardDataset(),
    2: TepKasperskyDataset(),
    3: SwatItrustDataset(),
}

In [5]:
for d in datasets:
    datasets[d].shake_not_stir()

# Preparing squads

In [6]:
squads = {
    0: WatchSquad(),
    1: WatchSquad(),
    2: WatchSquad(),
    3: WatchSquad(),
}

## you can fit ...

for d in tqdm(datasets):
    for data, _, _ in datasets[d].train_generator():
        squads[d].prefit(data)

for d in tqdm(datasets):
    for data, _, _ in datasets[d].train_generator():
        squads[d].fit(data)

for d in tqdm(datasets):
    for data, _, _ in datasets[d].train_generator():
        squads[d].postfit(data)

for d in tqdm(datasets):
    squads[d].dump(f'{datasets[d].__class__.__name__}')

## ... or just loading

In [7]:
for d in datasets:
    squads[d].load(f'{datasets[d].__class__.__name__}')

# Examine

In [8]:
metrics = ('precision', 'recall', 'f1_score')
results = pd.DataFrame(columns=metrics)

In [ ]:
for d in tqdm(datasets):
    exam_paper = pd.DataFrame(columns=metrics)
    for i_st, (data, faults, info) in enumerate(tqdm(datasets[d].test_generator())):
        detect = squads[d].predict(data)
        exam_paper.loc[i_st, metrics] = time_span_metrics(faults, detect)
    results.loc[str(datasets[d]), metrics] = exam_paper.mean().values

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/10500 [00:00<?, ?it/s]

In [ ]:
results